# Простейшая рекуррентная сеть
В этом ноутбуке мы пройдемся по основам работы с RNN. Сегодня займемся задачей генерации текста. 

In [517]:
import warnings
from typing import Iterable, Tuple
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.distributions.categorical import Categorical

warnings.filterwarnings("ignore")

В качестве обучающего датасета возьмем набор из 120 тысяч анекдотов на русском языке. 
[Ссылка на данные](https://archive.org/download/120_tysyach_anekdotov) и [пост на хабре про тематическое моделирование](https://habr.com/ru/companies/otus/articles/723306/)

In [518]:
with open(r"/home/an/Downloads/anek.txt", "r", encoding="utf-8") as f:
    text = f.read()
text[118:500]

'|startoftext|>Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!\n\n<|startoftext|>- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...\n\n<|startoftext|>- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От со'

Мы не хотим моделировать все подряд, поэтому разобьем датасет на отдельные анекдоты.  

In [519]:
def cut_data(text):
    return text.replace("\n\n", "").split("<|startoftext|>")[1:]

In [520]:
cut_text = cut_data(text)

In [521]:
cut_text[1:6]

['Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!',
 '- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...',
 '- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.',
 'Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.',
 'Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о тебе...']

Сделаем для начала самую простую модель с токенами на уровне символов. Это значит, что каждому символу в тексте ставится в соответствие некоторое число. Некоторые способы токенизации используют части слов или, наоборот, части бинарного представления текста.

In [522]:
unique_chars = tuple(set(text))
print(unique_chars)
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

('由', 'М', '给', ' ', 'у', '^', '已', 'С', '²', '»', 'ы', '副', '/', '\ufeff', '老', 'ъ', 'N', '`', 'E', 'O', 'm', 'Е', 'P', '€', 'h', '“', 'U', 'ш', '虽', '&', 'и', '\n', 'И', 'Б', '<', 'z', 'Я', 'ц', 'I', 'Ю', 'Г', '命', 'Р', 'Y', '̈', 'щ', '*', '经', 'X', '°', '”', 'в', 'G', 'π', 'а', '为', 'н', ':', 'ö', 'H', 'B', 'з', '−', 'ч', '应', ';', 'х', 'Ё', '́', '$', 'А', 't', 'Q', 'Ø', '3', '@', 'v', 'F', '0', 'i', 'Э', '7', '长', '¿', 'ο', 'э', 'К', 'u', '’', 'e', 'o', '的', '▒', '″', 'м', 'k', 'У', 'т', 'C', '=', '。', 'R', '2', 'V', 'Ъ', 'я', '.', '>', 'g', '名', 'L', 'M', 'Л', 'Н', '?', 'a', '成', 'ё', '̆', 'д', 'T', 'л', 'ф', 'Й', '举', '会', '选', 'З', 'J', 'Д', '#', 'П', 'Z', '结', '表', '6', '数', 'о', 's', 'б', '☺', '!', '-', 'j', '果', 'В', '9', 'Ч', 'ж', '×', '代', 'ю', '人', 'q', 'S', 'р', 'О', 'Ц', "'", 'е', 'Ж', '+', '事', 'Ь', '手', '理', '№', '然', 'Ф', 'd', 'ь', 'r', '新', 'p', '☻', 'ë', '接', '4', 'w', 'Х', ',', 'l', 'Ы', 'D', 'x', 'Ш', '8', '5', '_', '直', '|', 'K', '\u200b', 'й', 'y', 'n', 'A', 'с'

Напишем функции для энкодинга и декодинга нашего текста. Они будут преобразовывать список символов в список чисел и обратно.

In [523]:
def encode(sentence, vocab):
    l = []
    for ch in sentence:
        l.append(vocab[ch])
    return l

def decode(tokens, vocab):
    l = ""
    for tok in tokens:
        l += vocab[tok]
    return l

a = encode("sdfsdf asdfasdasdkslf sdfkh asdf", char2int)
print(a)
b = decode(a, int2char)
print(b)

[138, 169, 199, 138, 169, 199, 3, 115, 138, 169, 199, 115, 138, 169, 115, 138, 169, 95, 138, 181, 199, 3, 138, 169, 199, 95, 24, 3, 115, 138, 169, 199]
sdfsdf asdfasdasdkslf sdfkh asdf


Просто представления символов в виде числа не подходят для обучения моделей. На выходе должны быть вероятности всех возможных токенов из словаря. Поэтому модели удобно учить с помощью энтропии. К тому же, токены часто преобразуют из исходного представления в эмбеддинги, которые также позволяют получить более удобное представление в высокоразмерном пространстве. 

В итоге векторы в модели выглядят следующим образом:
![alt_text](../additional_materials/images/char_rnn.jfif)

Задание: реализуйте метод, который преобразует батч в бинарное представление.

In [524]:
def one_hot_encode(int_words: torch.Tensor, vocab_size: int) -> torch.Tensor:
    """Encodes batch of sentences into binary values"""
    batch_size, seq_len = int_words.shape
    words_one_hot = torch.zeros((batch_size, seq_len, vocab_size))
    
    for batch_idx in range(batch_size):
        words_one_hot[batch_idx, torch.arange(seq_len), int_words[batch_idx]] = 1.0
    return words_one_hot

Проверьте ваш код.

In [525]:
test_seq = torch.tensor([[2, 6, 4, 1], [0,3, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

tensor([[[0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


Однако, наши последовательности на самом деле разной длины. Как же объединить их в батч?

Реализуем два необходимых класса: 
- токенайзер, который будет брать текст, кодировать и декодировать символы. Еще одно, что будет реализовано там - добавлено несколько специальных символов (паддинг, конец последовательности, начало последовательности).
- Датасет, который будет брать набор шуток, используя токенайзер, строить эмбеддинги и дополнять последовательность до максимальной длины.

In [526]:
class Tokenizer:
    def __init__(self, text, max_len: int = 512):
        self.text = text
        self.max_len = max_len
        self.specials = ["<pad>", "<bos>", "<eos>"]
        self.unique_chars = list(set(text))
        self._add_special("<pad>")
        self._add_special("<bos>")
        self._add_special("<eos>")
        
        self.int2char = dict(enumerate(self.unique_chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
    
    def _add_special(self, symbol) -> None:
        if symbol not in self.unique_chars:
            self.unique_chars.append(symbol)

    @property
    def vocab_size(self):
        return len(self.unique_chars)
        
    def decode_symbol(self, el):
        return self.int2char[el]
        
    def encode_symbol(self, el):
        return self.char2int[el]
        
    def str_to_idx(self, chars):
        return [self.encode_symbol(ch) for ch in chars]

    def idx_to_str(self, idx):
        return [self.decode_symbol(i) for i in idx]

    def encode(self, chars):
        chars = ["<bos>"] + list(chars) + ["<eos>"]
        return self.str_to_idx(chars)

    def decode(self, idx):
        chars = self.idx_to_str(idx)
        return ''.join([ch for ch in chars if ch not in ["<bos>", "<eos>"]])


In [527]:
import torch
from torch.utils.data import Dataset

class JokesDataset(Dataset):
    def __init__(self, tokenizer, cut_text, max_len: int = 512):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.cut_text = cut_text
        self.pad_index = self.tokenizer.encode_symbol("<pad>")

    def __len__(self):
        return len(self.cut_text)
        
    def __getitem__(self, item):
        joke = self.cut_text[item]
        encoded_joke = self.tokenizer.encode(joke)

        if len(encoded_joke) > self.max_len:
            encoded_joke = encoded_joke[:self.max_len]

        padding_num = self.max_len - len(encoded_joke)
        padded_joke = encoded_joke + [self.pad_index] * padding_num

        return torch.tensor(padded_joke), len(encoded_joke)


In [528]:
tokenizer = Tokenizer(text)
dataset = JokesDataset(tokenizer, cut_text, 512)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Вопрос: А как бы мы должны были разделять данные на последовательности и батчи в случае, если бы использовался сплошной текст?

Теперь реализуем нашу модель. 
Необходимо следующее:
 - Используя токенайзер, задать размер словаря
 - Задать слой RNN с помощью torch.RNN. Доп.задание: создайте модель, используя слой LSTM.
 - Задать полносвязный слой с набором параметров: размерность ввода — n_hidden; размерность выхода — размер словаря. Этот слой преобразует состояние модели в логиты токенов.
 - Определить шаг forward, который будет использоваться при обучении
 - Определить метод init_hidden, который будет задавать начальное внутреннее состояние. Инициализировать будем нулями.
 - Определить метод inference, в котором будет происходить генерация последовательности из префикса. Здесь мы уже не используем явные логиты, а семплируем токены на их основе.


In [529]:
class CharRNN(nn.Module):
    def __init__(
        self,
        tokenizer,
        hidden_dim: int = 256,
        num_layers: int = 2,
        drop_prob: float = 0.5,
        max_len: int = 512,
    ) -> None:
        super().__init__()
        self.n_hidden = hidden_dim
        self.n_layers = num_layers
        self.max_len = max_len
        self.tokenizer = tokenizer
        
        vocab_size = self.tokenizer.vocab_size
        self.rnn = nn.RNN(input_size=vocab_size, hidden_size=self.n_hidden, num_layers=self.n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(self.n_hidden, vocab_size)

    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len = x.shape
        h0 = torch.zeros(self.n_layers, batch_size, self.n_hidden).to(device)

        one_hot = one_hot_encode(x, self.tokenizer.vocab_size)

        packed_embeds = pack_padded_sequence(one_hot, lengths, batch_first=True, enforce_sorted=False).to(device)
        packed_output, hidden = self.rnn(packed_embeds, h0)
        out, lengths = pad_packed_sequence(packed_output, batch_first=True)
        out = self.dropout(out)
        logits = self.fc(out)

        return logits, hidden

    def inference(self, prefix='<bos>'):
        tokens = torch.tensor(self.tokenizer.encode(prefix)).unsqueeze(0).to(device)

        while tokens.shape[1] < self.max_len:
            lengths = torch.tensor([tokens.shape[1]], dtype=torch.int64)  
            logits, hidden = self.forward(tokens, lengths)
            logits = logits[:, -1, :]

            new_token = torch.argmax(logits, dim=-1).unsqueeze(0)
            tokens = torch.cat([tokens, new_token], dim=1)

            if new_token.item() == self.tokenizer.encode_symbol('<eos>'):
                break

        return self.tokenizer.decode(tokens.squeeze().cpu().numpy())


Зададим параметры для обучения. Можете варьировать их, чтобы вам хватило ресурсов.

In [530]:
batch_size = 4
seq_length = 512
n_hidden = 64
n_layers = 4
drop_prob = 0.1
lr = 0.1

Напишите функцию для одного тренировочного шага. В этом ноутбуке сам процесс обучения модели достаточно тривиален, поэтому мы не будем использовать сложные функции для обучающего цикла. Вы же, однако, можете дописать их.

In [ ]:
def training_step(
    model: CharRNN,
    train_batch: Tuple[torch.Tensor, torch.Tensor],
    vocab_size: int,
    criterion: nn.Module,
    optimizer,
    device="cpu"
) -> torch.Tensor:
    optimizer.zero_grad()

    input_data, target_lengths = train_batch

    # Перемещение данных на устройство
    input_data = input_data.to(device)
    target_lengths = target_lengths.to(device)

    # Длина последовательностей
    batch_size, seq_len = input_data.shape

    # Получение выходов модели
    logits, _ = model(input_data, target_lengths)

    # Преобразование целевых данных и логитов
    logits = logits.view(-1, vocab_size)
    target_data = input_data.view(-1)

    # Удаление padding с учетом длины последовательностей
    mask = target_data != tokenizer.encode_symbol("<pad>")
    logits = logits[mask[:logits.size(0)]]  # Применяем маску только к размеру logits
    target_data = target_data[mask[:logits.size(0)]]

    # Проверки
    print(f"Logits after mask: {logits.shape}")
    print(f"Target data after mask: {target_data.shape}")

    # Вычисление ошибки
    loss = criterion(logits, target_data)

    # Обратное распространение
    loss.backward()
    optimizer.step()

    return loss

Инициализируйте модель, функцию потерь и оптимизатор.

In [532]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CharRNN(tokenizer, n_hidden, n_layers, drop_prob).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


Проверьте необученную модель: она должна выдавать бессмысленные последовательности

In [ ]:
model.eval()  

# prefix = "rtfdg"
prefix = "<bos>"
# prefix = "<eos>" 
generated_text = model.inference(prefix)
print("Output:", generated_text)


Output: 然。然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然然


In [534]:
def plot_losses(losses):
    clear_output()
    plt.plot(range(1, len(losses) + 1), losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

Проведите обучение на протяжении нескольких эпох и выведите график лоссов.

In [535]:
losses = []
num_epochs = 5

for epoch in range(1, num_epochs + 1):
    model.train()  
    epoch_loss = 0
        
    for batch_idx, train_batch in enumerate(dataloader):
        loss = training_step(model, train_batch, tokenizer.vocab_size, criterion, optimizer, device)
        epoch_loss += loss
        
        avg_epoch_loss = epoch_loss / len(dataloader)
        losses.append(avg_epoch_loss)
        
        print(f"Epoch {epoch}/{num_epochs}, Loss: {avg_epoch_loss:.4f}")
        
        plot_losses(losses)
        
        torch.save(model.state_dict(), f"rnn.pt")


IndexError: The shape of the mask [16384] at index 0 does not match the shape of the indexed tensor [8992, 217] at index 0

In [ ]:
[model.inference("") for _ in range(10)]

['1人☻Kшш的%sе的%П人的%Ш%KП%ш%ш/шПKшш的Ш%Kшш1%KWK>K%的K人шK的%П的人%%ш人的%的人%шШшш事Ks人Ш人ш1的%П的%K%ш的的手%Kш人е1ш人ш%1的的%%%K的人ншШWШ%ш成人K人的Ш的%KеKWsшK人%ш的1☻手K人1的N%%人1%ш%шПш%З%%人П手KП人%%ш的шK的%З人%ш的的人>手人人人%ш事的%П人шNш%人%ш人ш人ш的Ш%KsПшш%ш的ш人%ШШ%шKшsш%人ш人%шШЗ的手еПше的%的П人%шK手的%的%шПKшш人KKкЗ人ш1Шш%s人%人ш人шш事%1K1Kш人%%ш☻%Пш人%%ш%人%шешK的%Пшш%K1%%ш%%☻K人的Пs手人шеШ6ПшK%人的%s%人1шш人%%шш的%Пш%人е人Шш人sПШsш人шП的人пKшs人人1ПKшKЗшП的%%的人人1Шe%шП人Р人/шK的j%K<的人人%ше人ПШj的人1的ш人K人1%%шПшK%ш的%%K的%☻KШПш人ш1的%шш的%%的人ш%шШ人к人☻П的的人的%人%NШ人的%的%ш的П%人е1人е%ш的人人ш%的Ш人%%П的%人кш人人ФШKKK%шKшП人',
 '1人%K人%%П手шK人%%ш%ш%Пш人шш%шш人KKШ的人1П的人%%Ш的%ПLППк的人人ш%的人人Пs%%手шПs1%ш的人Ш人Ш人人%ш人шПшs的%е人人人ППKшsш%K的%ш1%事KшП人ш%1KшW手的N人的%的%KшKЗ人шШ的☻ШП%ш%ПW人%人%шш手人%1的的%%Ш的%%ШШшППKшsш1пш%人KП%K%ш人的人%%☻Пш%П人кшП的N%П人K的%П人6шш的人sП☻%K人шшK%K人ш手1%шкш人ш%人1并☻ш%%ШППшK的шK人Ш人ШKK%%Шш并ш%的%ш的>人%N人人的ШK人П%шШшK1人%шеш☻ПsшK人1人的%的%Ш人ш人K%的%的KK手Ш人K%Пш人KШ手П人的人☻%ш的%☻人的%ШеПшKKK%ш的手%K人K%%ШшПш人人ш>ш手K1人的%K%ш的人%еШшK%手1ш手ПK1N人ш人的%的手人ш☻%KШш%人%шш%人ш的的WK人%人人手ППк%人ш的%人шш☻人шKKе%Шнш☻1人%ш的人的%Ш的%ш人的ПП的Шш的П人%K人N并Ш1Kш%的%人K的ш人的ш的N%%人sNШ☻П人N人ШK

Теперь попробуем написать свой собственный RNN. Это будет довольно простая модель с одним слоем.


In [ ]:
# YOUR CODE: custom model nn.Module, changed CharRNN, etc